# Анализ результативности футбольных команд

Команда «Е.Д.А.»
 - Максимов Дмитрий Николаевич
 - Никитина Елизавета Андреевна
 - Соловьев Алексаднр Сергеевич 

## Шаг 1. Тема и общее описание проекта

В футболе существует огромное количество статистики по итогам матчей — количество пасов/навесов, xG (Expected Goals), доля ударов в створ и многое другое. Наша команда заинтересовалась, можно ли по этой статистике оценивать результативность футбольной команды и, соответственно, пытаться предсказывать исход матчей.

Для этого мы решили взглянуть на последние несколько сезонов Английской Премьер лиги и поставили задачу предугадать чемпиона текущего сезона. В качестве основного источника данных мы взяли сайт https://fbref.com/en/, так как он предоставляет большое количество статистики

## Шаг 2. Сбор данных

### Как мы собирали данные?
Fbref имеет некоторые ограничения для сбора данных — слишком частые запросы (больше 20 в минуту) блокируются, поэтому в одной из версий парсера мы решили расставить паузы так, чтобы не нарушать это требование со стороны fbref. В итоге необходимые данные мы скачали за примерно 1.5 часа. 
Код парсера можно посмотреть [здесь](https://github.com/asolovyov03/soccer-analysis/blob/main/scripts/parse.py)

Итоговые данные мы сохранили в файл [match_stats.json](https://github.com/asolovyov03/soccer-analysis/blob/main/match_stats.json)

### Какие данные мы собрали?
Мы решили взять статистику всех матчей АПЛ с сезона 2018-2019 по текущий (последний матч на момент парсинга прошёл 8 мая)
Для домашней и гостевой команды мы собрали следующие признаки:
- **team**: название команды
- **score**: количество голов, которая команда забила по итогам матча
- **xG**: показатель Expected Goals (расчитана аналитической компанией Opta)
- **possession**: доля владения мячом
- **passing_accuracy**: точность пасов (доля успешных пасов)
- **shots_on_target**: доля ударов в створ
- **yellow_cards_count**: количество желтых карточек
- **red_cards_count**: количество красных карточек
- **fouls**: количество совершенных фолов
- **corners**: количество угловых
- **crosses**: количество навесов
- **touches**: количество касаний мяча
- **tackles**: количество перехватов
- **interceptions**: количество отборов мяча
- **aerials_won**: количество выигрышей мяча в воздухе
- **clearances**: количество выбитых мячей
- **offsides**: количетсво попаданий в оффсайд
- **goal_kicks**: количество ударов от ворот
- **throw_ins**: количество вбросов в игру
- **long_balls**: количество дальних передач
- **match_date**: дата матча
- **match_week**: номер недели чемпионата

## Шаг 3. Предварительная обработка

## Шаг 4. Визуализация

## Шаг 5. Создание новых признаков

## Шаг 6. Гипотезы

## Шаг 7. Машинное обучение

**Описание, как мы собираемся это делать**

Так как почти все наши признаки известны лишь после матча, предсказывать результат напрямую по ним не выйдет. Наша идея состоит в том, чтобы агрегировать признаки каждой команды за последние n матчей.
n будет гиперпараметром, который вероятно придётся подбирать, так как сразу сказать, сколько матчей брать — сложно. Если взять очень маленький период, то фактор везения или случайности будет слишком сильно влиять на результат, а если очень большой, то какие-то данные могут быть устаревшими (со временем в команде может меняться тактика, тренер, состав и многое другое, что серьёзно влияет на её результативность).

Имея агрегированные данные двух команд мы собираемся посчитать разницу (или отношение) между ними и на основе этого обучить логистическую регрессию для предсказания вероятности победы домашней команды. Далее мы подберем пороги вероятностей для определения победы, ничьи и поражения.

Возможно, если качество модели логистической регрессии будет низкое, мы попробуем другие модели для задачи многоклассовой классификации (например, деревья или бустинг), но пока это в наши планы не входит.